In [19]:
import tensorflow.compat.v1 as tensorflow
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU, Activation, Flatten, Dense, Lambda
from tensorflow.keras.layers import Reshape, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [63]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from keras.datasets import cifar10

In [64]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [80]:
img = Input(shape=(32,32,3))
Latent_size = 2

In [81]:
# creating encoder
x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(img)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

shape_bfal = x.shape[1:]
x = Flatten()(x)

mu = Dense(Latent_size, name='mu')(x)
log_var = Dense(Latent_size, name='log_var')(x)

encoder_z = Model(img, (mu, log_var))

In [82]:
print(shape_bfal)

(8, 8, 64)


In [83]:
encoder_z.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_13 (Conv2D)             (None, 32, 32, 32)   896         ['input_7[0][0]']                
                                                                                                  
 batch_normalization_12 (BatchN  (None, 32, 32, 32)  128         ['conv2d_13[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 leaky_re_lu_20 (LeakyReLU)     (None, 32, 32, 32)   0           ['batch_normalization_12[0

In [84]:
def sampling(args):
    mu, log_var = args
    epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1.)
    return mu + K.exp(log_var/2)*epsilon

In [85]:
encoder_output = Lambda(sampling, name='encoder_output')([mu, log_var])
encoder = Model(img, encoder_output)

In [96]:
# creating decoder
decoder_input = Input(shape=(Latent_size), name='decoder_input')
x = Dense(np.prod(shape_bfal))(decoder_input)
x = Reshape(shape_bfal)(x)

x = Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(filters=3, kernel_size=3, strides=1, padding='same')(x)
x = Activation('sigmoid')(x)

decoder_output = x
decoder = Model(inputs=decoder_input, outputs=decoder_output)

In [97]:
decoder_output.shape

TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(3)])

In [98]:
decoder.summary()

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_5 (Dense)             (None, 4096)              12288     
                                                                 
 reshape_5 (Reshape)         (None, 8, 8, 64)          0         
                                                                 
 conv2d_transpose_15 (Conv2D  (None, 16, 16, 64)       36928     
 Transpose)                                                      
                                                                 
 leaky_re_lu_25 (LeakyReLU)  (None, 16, 16, 64)        0         
                                                                 
 conv2d_transpose_16 (Conv2D  (None, 32, 32, 32)       18464     
 Transpose)                                               

In [99]:
varAE = Model(inputs=img, outputs=decoder(encoder_output))

In [100]:
# # creating loss functions
def vae_r_loss(y_true,y_pred):
    r_loss = K.mean(K.square(y_true-y_pred), axis=1)
    return r_loss

def vae_kl_loss(y_true, y_pred):
    kl_loss = -0.5*K.sum(1 + log_var - K.square(mu) - K.exp(log_var),axis=1)
    return kl_loss

def vae_loss(y_true, y_pred):
    r_loss = vae_r_loss(y_true, y_pred)
    kl_loss = vae_kl_loss(y_true, y_pred)
    return r_loss + kl_loss

varAE.compile(optimizer='adam', loss=vae_loss)

In [101]:
# #implementing a custom loss is now a headache
# class Custom_CE_Loss(tensorflow.keras.losses.loss):
#     def __init__(self):
#         super().__init__()
#     def call(self, y_true, y_pred):
#         r_loss = vae_r_loss(y_true, y_pred)
#         kl_loss = vae_kl_loss(y_true, y_pred)
#         return r_loss + kl_loss

In [102]:
# dataset = tensorflow.constant(x_train)

In [103]:
import tensorflow._api.v2.compat.v1 as tf

tf.disable_v2_behavior()

In [104]:
varAE.fit(x_train,x_train, epochs=50, shuffle=True)

Train on 50000 samples
Epoch 1/50


InvalidArgumentError: Incompatible shapes: [32,32,3] vs. [32]
	 [[{{node training_2/Adam/gradients/gradients/loss_5/model_20_loss/add_1_grad/BroadcastGradientArgs}}]]